In [28]:
from fractions import Fraction

In [29]:
#This function takes as an argument vector k=k1,k1 and throws out associated constant theta_hat

theta_hat = {
    "*": Fraction(-3,2**3),
    "0": Fraction(1,2**2),
    "|k|=1": Fraction(-1,2**3), 
    "prod_k_p=+1": Fraction(-1,2**5),
    "prod_k_p=-1": Fraction(5,2**5)
}

def theta_hat_eval(k):
    if (k[0]==k[1]==0):
        return(theta_hat["0"])
    if k[0]**2+k[1]**2==1:
        return(theta_hat["|k|=1"])
    if (k[0]*k[1]==1):
        return(theta_hat["prod_k_p=+1"])
    if (k[0]*k[1]==-1):
        return(theta_hat["prod_k_p=-1"])

In [30]:
from sympy import sin, cos
from sympy.abc import x
from sympy import integrate
from sympy import *

def psi_hat(a,b):
    return(integrate(cos(x)**(2*a)*sin(x)**(2*b)/pi, (x,0,2*pi)))

In [31]:
def cov(y,nu):
    y = list(y)
    norm = sum(map(lambda z:z*z,y))
    positive = 0
    negative = 0  
    for m in range(len(y)):
        if y[m]>0:
            positive = positive + 1
        if y[m]<0: 
            negative = negative + 1
    const = Fraction(1,2**2)*(2**(positive+negative))*(2+(-1)**norm*nu)*psi_hat(negative,positive)
    return(const)

In [32]:
#Computing covariances remaining terms
def cov_star(y,mu):
    if y=="*":
        const = (2+mu)*4*psi_hat(2,2) 
    else:
        norm = 0
        norm = y[0]**2+y[1]**2
        positive = 0
        negative = 0
        for m in range(2):
            if y[m]>0:
                positive = positive + 1
            if y[m]<0: 
                negative = negative + 1
        const = (2**norm)*(2+(-1)**norm*mu)*psi_hat(1+positive,1+negative)
    return(const)

In [33]:
def var_b_E(mu):
    #for some strange (unknonw) technical (python) reason import product needs to be here
    #and not at the begining of a program
    from itertools import product
    
    #var1 := \sum_{k,l} \theta_hat(k)*\theta_hat(l)*cov(\widehat{b_{\EEv}}^k,\widehat{b_{\EEv}}^l)
    var1 = 0 
    
    for y in list(product([-1,0,1],repeat = 4)):
        z = list(y)
        var1 = var1 + theta_hat_eval([z[0],z[1]])*theta_hat_eval([z[2],z[3]])*cov(y,mu)
    
    #var2 := cov(\widehat{b_{\EEv}},b_{\EEv}^*)
    var2 = 0
    
    for y in list(product([-1,0,1],repeat = 2)):
        y = list(y)
        var2 = var2+theta_hat_eval(y)*cov_star(y,mu)
    
    var2 = var2*2*theta_hat["*"]

    var3 = (theta_hat["*"]**2)*cov_star("*",mu) #var3 := Var(b_E^*)
    
    var = var1 + var2 + var3
    
    return(var)
    


In [34]:
print("if mu=1 we recover one-energy case",var_b_E(1))
print("if mu=0 we obtain different value",var_b_E(0))

if mu=1 we recover one-energy case 43/128
if mu=0 we obtain different value 9/64


In [35]:
#The remaining code is only for testing purposes
#it recovers partial results given in [NPR19]

In [36]:
def decoding_to_npr19(w):
    if w == 1:
        return([0,0])
    if w == 2:
        return([0,-1])
    if w == 3:
        return([0,1])
    if w == 4:
        return([-1,0])
    if w == 5:
        return([1,0])
    if w == 6:
        return([-1,-1])
    if w == 7:
        return([1,1])
    if w == 8:
        return([-1,1])
    if w == 9:
        return([1,-1])
    if w == 10:
        return("*") 

In [37]:
#Printing partial results for main part under
#matched notation on [NPR19], beware of the fact 
#that partial results for cov(b_E^k,b_E^l) in [NPR19] 
#are smaller by a factor of 4 than they should be so
#that's why we had to divide by a factor of 4. 

#for some strange (unknonw) technical (python) reason import product needs to be in the
#same cell where we use it
from itertools import product

for q in list(product([1,2,3,4,5,6,7,8,9],repeat = 2)):
    if q[0] < q[1]+1:
        decoded = decoding_to_npr19(q[0])+decoding_to_npr19(q[1])
        print(q,cov(decoded,1)/4) #fix mentioned at beginnig of the cell

(1, 1) 3/8
(1, 2) 1/8
(1, 3) 1/8
(1, 4) 1/8
(1, 5) 1/8
(1, 6) 9/16
(1, 7) 9/16
(1, 8) 3/16
(1, 9) 3/16
(2, 2) 9/16
(2, 3) 3/16
(2, 4) 9/16
(2, 5) 3/16
(2, 6) 5/16
(2, 7) 1/16
(2, 8) 1/16
(2, 9) 1/16
(3, 3) 9/16
(3, 4) 3/16
(3, 5) 9/16
(3, 6) 1/16
(3, 7) 5/16
(3, 8) 1/16
(3, 9) 1/16
(4, 4) 9/16
(4, 5) 3/16
(4, 6) 5/16
(4, 7) 1/16
(4, 8) 1/16
(4, 9) 1/16
(5, 5) 9/16
(5, 6) 1/16
(5, 7) 5/16
(5, 8) 1/16
(5, 9) 1/16
(6, 6) 105/64
(6, 7) 9/64
(6, 8) 15/64
(6, 9) 15/64
(7, 7) 105/64
(7, 8) 15/64
(7, 9) 15/64
(8, 8) 9/64
(8, 9) 9/64
(9, 9) 9/64


In [40]:
#The same comments on import product and
#correction by 1/4 apply. 

from itertools import product

for y in list(product([-1,0,1],repeat = 2)):
    print(y,cov_star(y,1)/4)

(-1, -1) 15/64
(-1, 0) 1/16
(-1, 1) 9/64
(0, -1) 1/16
(0, 0) 3/16
(0, 1) 1/16
(1, -1) 9/64
(1, 0) 1/16
(1, 1) 15/64


In [43]:
#The same comment about correction by 1/4 applies.
cov_star("*",1)/4

9/64